<h3>Vytvoření y</h3>

y = co má neuronka předpovídat. V našem případě začneme tím, zda byl v daném čase L-mod nebo H-mod.

Je třeba:

- načíst signály popisující začátek a konec H-módu v daném výboji: `t_H_mode_start` a `t_H_mode_end`.

- z adresáře s obrazovými daty zjistit časy snímků kamery v daném výboji

- pro každý čas zjistit, zda jde o L-mód nebo H-mód

- uložit do csv (např ve formátu slupců [název souboru se snímkem z kamery, čas ve výboji, label zda L-mode/H-mode]) - použijte pandas

Importing libraries

In [ ]:
import os
os.chdir("/compass/Shared/Users/bogdanov/vyzkumny_ukol")

In [ ]:
from cdb_extras import xarray_support as cdbxr   # načítání dat z databáze COMPASSu
from pyCDB import client
import numpy as np         # práce s numerickými poli
import pandas as pd
from pathlib import Path
%matplotlib inline

#import module with functions from the tutorial
import cdb_img_processing as img
cdb = client.CDBClient()

Importing data

In [9]:
shot = 16534
t_ELM_start = cdb.get_signal(f"t_ELM_start/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_ELM_end = cdb.get_signal(f"t_ELM_end/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_H_mode_start = cdb.get_signal(f"t_H_mode_start/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_H_mode_end = cdb.get_signal(f"t_H_mode_end/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")

In [10]:
ris1_data = img.load_RIS_data(shot, 1)
ris2_data = img.load_RIS_data(shot, 2)

ris1_imgs = img.flip_image(img.demosaic(ris1_data)[1][:34])    #[]<==test REMove after completion the rest of the task) 
ris2_imgs = img.flip_image(img.demosaic(ris2_data)[1][:34])

In [14]:
#TODO:  To create a DataFrame with only one row, you need to specify an index, so if during the shot plasma was in H-mode more than once...
t_H_mode = pd.DataFrame({'start':t_H_mode_start.data, 'end':t_H_mode_end.data}, index=[0])
t_ELM = pd.DataFrame({'start':t_ELM_start.data, 'end':t_ELM_end.data})

In [15]:
LorH = pd.DataFrame(data={'mode':np.full(len(ris1_imgs), 1)}, index=pd.Index(ris1_imgs.time, name='time'))#contains time and the state of the plasma (L-mode, H-mode, ELM)
# 'mode'=1 stands for L-mode, 'mode'=2 stands for  H-mode, 'mode'=3 stands for  => ELM

In [16]:
LorH

,mode
time,
950.0,1
950.2,1
950.4,1
950.6,1
950.8,1
...,...
1395.4,1
1395.6,1
1395.8,1


In [17]:
#Nevim jak bych to udelal bez for cyklu:
for elm in t_ELM.values:
    LorH.loc[elm[0]:elm[1]] = 3   #1 will stand for L-mode, 2 for H-mode and 3 for ELM

for H_mode in t_H_mode.values:
    LorH.loc[H_mode[0]:H_mode[1]] = 2 #1 will stand for L-mode, 2 for H-mode and 3 for ELM

Saving the images

In [18]:
out_path = Path('./imgs')

ris1_names = img.save_ris_images_to_folder(shot, out_path, ris=1) #I conciously decided not to cut the time in the filename at any decimal

0/2232 imgs are saved
50/2232 imgs are saved
100/2232 imgs are saved
150/2232 imgs are saved
200/2232 imgs are saved
250/2232 imgs are saved
300/2232 imgs are saved
350/2232 imgs are saved
400/2232 imgs are saved
450/2232 imgs are saved
500/2232 imgs are saved
550/2232 imgs are saved
600/2232 imgs are saved
650/2232 imgs are saved
700/2232 imgs are saved
750/2232 imgs are saved
800/2232 imgs are saved
850/2232 imgs are saved
900/2232 imgs are saved
950/2232 imgs are saved
1000/2232 imgs are saved
1050/2232 imgs are saved
1100/2232 imgs are saved
1150/2232 imgs are saved
1200/2232 imgs are saved
1250/2232 imgs are saved
1300/2232 imgs are saved
1350/2232 imgs are saved
1400/2232 imgs are saved
1450/2232 imgs are saved
1500/2232 imgs are saved
1550/2232 imgs are saved
1600/2232 imgs are saved
1650/2232 imgs are saved
1700/2232 imgs are saved
1750/2232 imgs are saved
1800/2232 imgs are saved
1850/2232 imgs are saved
1900/2232 imgs are saved
1950/2232 imgs are saved
2000/2232 imgs are save

In [ ]:
pwd

'/compass/Shared/Users/bogdanov/vyzkumny_ukol'

In [ ]:
type(ris1_names)

In [10]:
LorH['ris1'] = ris1_names  #- BAD IDEA

In [ ]:
ris1_names

In [12]:
LorH

,mode,ris1
time,,
950.0,1,None
950.2,1,None
950.4,1,None
950.6,1,None
950.8,1,None
...,...,...
1395.4,1,None
1395.6,1,None
1395.8,1,None


In [11]:
ris2_names = save_ris_images_to_folder(shot, out_path, ris=2)

NameError: name 'save_ris_images_to_folder' is not defined

In [ ]:
LorH['ris2'] = ris2_names  #- BAD IDEA

In [ ]:
LorH.to_csv(f'/compass/Shared/Users/bogdanov/vyzkumny_ukol/LHmode-detection-shot{shot}.csv')